In [8]:
import mlflow
import mlflow.sklearn
import joblib
import datetime
import numpy as np
import pandas as pd
import os
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from mlflow.models import infer_signature

# Set MLflow tracking URI
MLFLOW_TRACKING_URI = "http://127.0.0.1:5000/"
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

# Load the latest model from MLflow
MODEL_URI = "models:/linear_regression_model/latest"
model = mlflow.pyfunc.load_model(MODEL_URI)

# Load original training dataset
df_original = pd.read_csv("../data/processed/2024/train.csv", parse_dates=['Trip_Start', 'Trip_End'])

# Load stored new data from API inputs
new_data_file = "../api/saved_inputs/prediction_inputs.csv"

if os.path.exists(new_data_file):
    df_new = pd.read_csv(new_data_file)
    print(f"Loaded {len(df_new)} new data points for retraining.")
else:
    df_new = None
    print("No new data found. Skipping retraining.")

# Sample 10% of original data for fairness
df_original = df_original.sample(frac=0.1, random_state=42)

# Feature Engineering on original dataset
df_original['Trip_Duration'] = (df_original['Trip_End'] - df_original['Trip_Start']).dt.total_seconds()
df_original['Start_Hour'] = df_original['Trip_Start'].dt.hour
df_original['Start_DayOfWeek'] = df_original['Trip_Start'].dt.dayofweek

df_original = df_original.dropna(subset=['Trip_Duration', 'Year_of_Birth', 'Gender', 'Origin_Id', 'Destination_Id'])

c:\Users\ivanr\AppData\Local\anaconda3\envs\l73.10\lib\site-packages\mlflow\store\artifact\utils\models.py:31: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


Loaded 13 new data points for retraining.


In [9]:
df_original.head()

,Trip_Id,User_Id,Gender,Year_of_Birth,Trip_Start,Trip_End,Origin_Id,Destination_Id,Trip_Duration,Start_Hour,Start_DayOfWeek
3825046,36217695,4052665,1,1972.0,2024-08-31 23:59:16,2024-09-01 00:08:19,201,195,543.0,23,5
4059787,36485768,132245,1,1979.0,2024-11-12 08:47:14,2024-11-12 08:58:41,249,82,687.0,8,1
2667596,34900510,620240,1,1996.0,2024-07-28 11:11:52,2024-07-28 11:42:16,48,94,1824.0,11,6
1188885,33170807,2563762,1,1967.0,2024-04-02 18:58:31,2024-04-02 19:16:39,196,52,1088.0,18,1
2680072,34914717,1173753,1,1974.0,2024-07-29 16:53:25,2024-07-29 17:17:25,11,11,1440.0,16,0


In [10]:
df_new.head()

,Year_of_Birth,Gender,Origin_Id,Destination_Id,Start_Hour,Start_DayOfWeek,Prediction,Timestamp
0,1996.0,1,1,50,14,2,598.081141,2025-01-29 16:47:56
1,1996.0,1,1,50,14,2,598.081141,2025-01-29 16:48:00
2,1996.0,1,1,50,14,2,598.081141,2025-01-29 16:48:52
3,1996.0,1,1,50,14,2,598.081141,2025-01-29 16:48:54
4,1996.0,1,1,50,14,2,598.081141,2025-01-29 16:54:49


In [11]:
# **Standardizing `df_new` to match `df_original`**
if df_new is not None:
    # Rename 'Prediction' to 'Trip_Duration'
    df_new = df_new.rename(columns={'Prediction': 'Trip_Duration'})

    # Add missing columns with placeholders
    df_new['Trip_Id'] = -1  # Dummy ID
    df_new['User_Id'] = -1  # Dummy User

    # **Reconstruct `Trip_Start` based on Start_Hour & Start_DayOfWeek**
    current_year = datetime.datetime.now().year
    df_new['Trip_Start'] = df_new.apply(
        lambda row: pd.Timestamp(
            year=current_year,
            month=1,  # Placeholder month
            day=row['Start_DayOfWeek'] + 1,  # Assuming Monday=0, Sunday=6
            hour=row['Start_Hour']
        ),
        axis=1
    )

    # **Calculate `Trip_End`**
    df_new['Trip_End'] = df_new['Trip_Start'] + pd.to_timedelta(df_new['Trip_Duration'], unit='s')

    # Reorder columns to match `df_original`
    df_new = df_new[df_original.columns]

    # Concatenate original and new data
    df_combined = pd.concat([df_original, df_new], ignore_index=True)

    # Drop rows where `Trip_Duration` is NaN
    df_combined = df_combined.dropna(subset=['Trip_Duration'])

else:
    df_combined = df_original



In [13]:
df_combined

,Trip_Id,User_Id,Gender,Year_of_Birth,Trip_Start,Trip_End,Origin_Id,Destination_Id,Trip_Duration,Start_Hour,Start_DayOfWeek
0,36217695,4052665,1,1972.0,2024-08-31 23:59:16,2024-09-01 00:08:19.000000000,201,195,543.000000,23,5
1,36485768,132245,1,1979.0,2024-11-12 08:47:14,2024-11-12 08:58:41.000000000,249,82,687.000000,8,1
2,34900510,620240,1,1996.0,2024-07-28 11:11:52,2024-07-28 11:42:16.000000000,48,94,1824.000000,11,6
3,33170807,2563762,1,1967.0,2024-04-02 18:58:31,2024-04-02 19:16:39.000000000,196,52,1088.000000,18,1
4,34914717,1173753,1,1974.0,2024-07-29 16:53:25,2024-07-29 17:17:25.000000000,11,11,1440.000000,16,0
...,...,...,...,...,...,...,...,...,...,...,...
430593,-1,-1,1,1996.0,2025-01-03 14:00:00,2025-01-03 14:09:58.081140661,1,50,598.081141,14,2
430594,-1,-1,0,1996.0,2025-01-03 14:00:00,2025-01-03 14:10:24.444127631,1,50,624.444128,14,2
430595,-1,-1,1,1996.0,2025-01-03 14:00:00,2025-01-03 14:09:58.081140661,1,50,598.081141,14,2
430596,-1,-1,1,1996.0,2025-01-03 14:00:00,2025-01-03 14:09:58.081140661,1,50,598.081141,14,2


In [14]:
# **Define Features and Target**
X = df_combined[['Year_of_Birth', 'Gender', 'Origin_Id', 'Destination_Id', 'Start_Hour', 'Start_DayOfWeek']]
y = df_combined['Trip_Duration']

# Ensure correct data types
X = X.astype({
    "Year_of_Birth": float,
    "Gender": int,
    "Origin_Id": int,
    "Destination_Id": int,
    "Start_Hour": int,
    "Start_DayOfWeek": int
})

# Preprocessing pipeline
categorical_features = ['Gender']
numerical_features = ['Year_of_Birth', 'Origin_Id', 'Destination_Id', 'Start_Hour', 'Start_DayOfWeek']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ]
)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define model pipeline
lr_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Evaluate current model performance
y_pred_existing = model.predict(X_test)
mae_existing = mean_absolute_error(y_test, y_pred_existing)

# Define degradation threshold
THRESHOLD = 5.0  # Adjust based on acceptable performance drop


In [15]:
# If performance degrades, retrain
if mae_existing > THRESHOLD:
    print(f"Performance degraded (MAE: {mae_existing:.4f}). Retraining model...")

    mlflow.set_experiment("trip_duration_prediction")

    # **Generate timestamp for run name**
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    run_name = f"linear_regression_retrain_{timestamp}"

    with mlflow.start_run(run_name=run_name):
        # Train new model
        lr_pipeline.fit(X_train, y_train)
        y_pred_new = lr_pipeline.predict(X_test)

        # Compute new metrics
        mae_new = mean_absolute_error(y_test, y_pred_new)
        rmse_new = np.sqrt(mean_squared_error(y_test, y_pred_new))

        # Infer schema
        signature = infer_signature(X_train, y_pred_new)

        # Log new metrics
        mlflow.log_metric("mae", mae_new)
        mlflow.log_metric("rmse", rmse_new)

        # Save new model
        mlflow.sklearn.log_model(lr_pipeline, "linear_regression_model", signature=signature)

        print(f"New model retrained and logged (MAE: {mae_new:.4f}).")

    # Delete old input data to avoid duplicate training
    os.remove(new_data_file)
    print("Old input data deleted.")
else:
    print(f"No retraining needed. Current MAE: {mae_existing:.4f} is within acceptable range.")

Performance degraded (MAE: 284.7796). Retraining model...


c:\Users\ivanr\AppData\Local\anaconda3\envs\l73.10\lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


New model retrained and logged (MAE: 285.0600).
🏃 View run linear_regression_retrain_20250129_181907 at: http://127.0.0.1:5000/#/experiments/507713236911629796/runs/d688a3f45b784caaa785f6891c3b29f4
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/507713236911629796
Old input data deleted.
